# Database Options for an ASL Translator Microservice Application: A Comparative Analysis

## 1. Project Context and Data Requirements

Our project is a microservice-based American Sign Language (ASL) translator, consisting of the following components:

- **API service**: A FastAPI application serving a web front-end and handling prediction requests.
- **Inference service**: A Python-based model worker that processes images asynchronously via RabbitMQ.
- **Message broker**: RabbitMQ, used for asynchronous communication between services.

The data our application currently handles, or is likely to handle in the future, includes:

- **Prediction logs**: Job ID, predicted gesture, confidence score, timestamp, latency, and potentially a user ID.
- **User data**: User accounts, roles, preferences, and usage history.
- **Feedback data**: User feedback indicating whether a prediction was correct or incorrect, including possible manual corrections.
- **Model metadata**: Model versions, training timestamps, and performance metrics.
- **Image data**: Either raw image data or references/paths to stored images.

From these requirements, our database solution must support:

- Structured, relational data (e.g., predictions, users, feedback).
- Strong consistency to avoid conflicting states between predictions and feedback.
- Good integration with Python/FastAPI and easy Docker-based deployment.
- Potential extension toward analytics, monitoring, and model evaluation.

---

## 2. Database Families Considered

To determine the most suitable database solution for our project, we compared several database families:

- **Relational SQL databases**: PostgreSQL, MySQL/MariaDB, SQLite.
- **Document databases**: MongoDB.
- **Wide-column / distributed databases**: Apache Cassandra.
- **In-memory / cache stores**: Redis.
- **Time-series databases**: InfluxDB and TimescaleDB.
- **Vector databases (future-oriented)**: PostgreSQL with pgvector and dedicated vector databases such as Pinecone.

---

## 3. Relational SQL Databases

### 3.1 PostgreSQL

#### Why PostgreSQL is a strong candidate

PostgreSQL aligns well with the relational nature of our data. Entities such as predictions, users, feedback, and models have clear relationships (one-to-many and many-to-many), which can be naturally modeled in relational tables.

It also supports complex analytical queries, such as calculating average confidence per gesture per model version over a given time period, which is useful for monitoring model performance.

PostgreSQL is a mature and reliable system, offering ACID-compliant transactions and strong consistency guarantees. It is widely used in production environments and benefits from robust tooling for backups, monitoring, and maintenance.

From a development perspective, PostgreSQL integrates very well with FastAPI through tools such as SQLAlchemy and Alembic. There are numerous examples and tutorials for deploying FastAPI applications with PostgreSQL in Docker.

Additionally, PostgreSQL supports advanced analytics features such as powerful indexing, window functions, and JSONB columns for semi-structured data. Extensions like TimescaleDB and pgvector allow PostgreSQL to be used as a hybrid time-series or vector database in the future.

Finally, PostgreSQL is Docker-friendly, with official images that can be easily deployed alongside our microservices.

#### Arguments against PostgreSQL

Compared to SQLite, PostgreSQL introduces some operational overhead, as it requires running and managing a separate service. For very small or purely local projects, this may be unnecessarily complex. However, for our containerized microservice architecture, PostgreSQL remains a practical and scalable choice.

---

### 3.2 MySQL / MariaDB

#### Pros

MySQL and MariaDB are mature relational databases with strong transaction support (via InnoDB). They provide similar benefits to PostgreSQL in terms of structured data, joins, and indexes. Both databases also have good Docker support and Python connectors.

#### Cons / Why PostgreSQL is usually preferred

MySQL and MariaDB are slightly less feature-rich than PostgreSQL, particularly for complex analytics, JSON handling, and advanced extensions. In addition, current Python/FastAPI communities and tutorials favor PostgreSQL as the default choice.

**Conclusion**: MySQL/MariaDB are viable options, but PostgreSQL is generally preferred unless there is an existing MySQL-based infrastructure.

---

### 3.3 SQLite

#### Pros

SQLite is extremely simple to set up, as it is file-based and does not require a separate server process. It supports full SQL and ACID compliance for single-node scenarios, making it ideal for quick local prototyping and unit testing. Its lightweight nature allows it to be easily bundled with the application.

#### Cons

SQLite is not ideal for multi-container microservice deployments because file-based databases do not scale well across containers. Concurrency is limited, and reliability can be an issue if multiple services attempt simultaneous writes. Migrating to a production database later may require code changes.

**Conclusion**: SQLite is suitable for experiments and local development, but not recommended as the main database in a Dockerized microservice architecture like ours.

---

## 4. Document Databases

### 4.1 MongoDB

#### Potential advantages

MongoDB offers a flexible schema, allowing us to store prediction results as JSON documents with variable fields. This flexibility is useful if our data evolves quickly or contains nested structures, such as multiple candidate predictions or model metadata in a single document.

MongoDB can feel natural for developers who think in JSON. It also provides built-in support for horizontal scaling, sharding, and high availability.

#### Arguments against MongoDB for our project

Our core data is naturally relational. Relationships such as **users ↔ predictions ↔ feedback** are more efficiently and robustly handled in a relational database like PostgreSQL.  

While modern MongoDB supports multi-document transactions, SQL databases remain more straightforward for relational, transactional workloads. Running a MongoDB cluster also introduces operational overhead that may be unnecessary for a small group project.

**Conclusion**: MongoDB is viable, especially if schema flexibility is important, but PostgreSQL is generally a better fit for our core data.

---

## 5. Wide-Column / Distributed Databases

### 5.1 Apache Cassandra

#### Pros

Cassandra is designed for high write throughput and horizontal scalability, making it suitable for extremely large datasets. It also provides fault tolerance across multi-node and multi-datacenter deployments.

#### Cons

For our project, Cassandra’s operational complexity is a significant drawback. Deployment, tuning, and maintenance are challenging, especially for a small team. Its data model requires careful design around query patterns, making ad-hoc queries less natural than in SQL databases. Moreover, the scale Cassandra is designed for exceeds the needs of our ASL translator.

**Conclusion**: Cassandra is technically interesting but not recommended for this project due to its complexity and mismatch with our current requirements.

---

## 6. In-Memory and Cache Stores

### 6.1 Redis

#### Best use-cases for our project

- Caching recent predictions to speed up repeated requests.
- Storing short-lived data such as session tokens, rate-limiting counters, or temporary job states.

#### Arguments for Redis

- Extremely fast reads and writes, as it is in-memory.
- Simple key-value model, supporting lists, sets, and sorted sets.
- Excellent Docker support and Python client libraries.

#### Arguments against using Redis as the main DB

- Not intended as a primary data store.
- Persistence is optional and may risk data loss after a restart or failure.
- Limited support for complex queries, joins, or analytics.

**Conclusion**: Redis is suitable as a secondary component (cache or temporary storage), but not as a replacement for a relational database.

---

## 7. Time-Series Databases

### 7.1 InfluxDB / TimescaleDB

#### Potential uses

- Monitoring and observability: tracking request counts, latency distributions, error rates, and model performance over time.
- Model performance analytics: analyzing confidence and accuracy trends per model version.

#### Arguments for

- Optimized for time-series data with continuous queries, downsampling, and retention policies.
- Integrates well with visualization tools like Grafana.

#### Arguments against as the main DB

- Not ideal for relational data such as users, predictions, and feedback.
- Adds complexity to the architecture.

**Conclusion**: Useful for monitoring or analytics at scale. For initial development, time-series metrics can be stored in PostgreSQL or Prometheus.

---

## 8. Vector Databases (Future-Oriented)

If we later extend the ASL translator to:

- Store gesture embeddings, or  
- Perform semantic search (e.g., find similar gestures or map gestures to concepts),

then a vector database could be useful. Examples include PostgreSQL with **pgvector**, Qdrant, Milvus, Pinecone, or Weaviate.

Vector databases allow k-NN (nearest neighbor) searches in embedding space. For now, starting with PostgreSQL is sufficient, with the option to add pgvector or a separate vector DB later.

---

## 9. Comparative Summary

| Database | Why | Why Not |
|----------|-----|---------|
| PostgreSQL | Best match for relational, transactional data. Strong FastAPI ecosystem. Good for analytics and Docker. | Slightly more setup than SQLite; overkill for small local-only prototypes. |
| MySQL / MariaDB | Mature SQL DB; structured data, joins, indexes. | Fewer advanced features than PostgreSQL; Python/FastAPI examples favor PostgreSQL. |
| SQLite | Very simple, file-based; ideal for quick local development/tests. | Not ideal for multi-container microservices; concurrency limits; not suitable long-term. |
| MongoDB | Flexible schema; good for semi-structured JSON; evolves easily. | Core data is relational; SQL better for transactions/joins; extra operational overhead. |
| Cassandra | High write throughput; fault-tolerant; massive scale. | Overkill for our scale; operationally complex; ad-hoc queries less natural. |
| Redis | Great for caching, temporary states, sessions. | Not durable; limited querying; not suitable as primary DB. |
| InfluxDB / TimescaleDB | Optimized for time-series metrics; monitoring; analytics. | Not suitable for relational core data; adds complexity. |
| Vector DB | Useful if we store embeddings for semantic search. | Overkill for basic gesture classification and logging. |

---

## 10. Recommendation for Our ASL Translator Project

**Primary database (core choice):**

- PostgreSQL for:
  - Prediction logs (job ID, gesture, confidence, latency, timestamp, user, model version)
  - Users and authentication data
  - Feedback and corrections
  - Model metadata and experiment tracking (basic level)

**Secondary components (optional, as the project grows):**

- Redis for caching and short-lived data.
- Prometheus + Grafana, or PostgreSQL time-series tables, for metrics.
- pgvector or a vector database if embeddings and similarity search are introduced.

**Why this combination fits best:**

- Aligns with our microservice and Docker architecture.
- Matches our Python/FastAPI stack and common tutorials.
- Balances simplicity (one main DB to learn and maintain) with flexibility (extensions for time-series and vector search).
- Provides a solid foundation for both short-term coursework and potential long-term project evolution.

## 11. Handling Image Data for Training and Testing

### 11.1 Storage Options

For our project, image data used in training and testing can be stored in two main ways:

1. **As files in a filesystem or cloud storage, with references in the database**  
   - Images are saved in a directory structure or cloud bucket (e.g., AWS S3, GCP Cloud Storage).  
   - The database (PostgreSQL) stores **paths or URLs** to the images, along with metadata such as gesture label, user ID, timestamp, and dataset split (train/test/validation).  

   **Advantages:**  
   - Keeps the database small and efficient.  
   - Easy to scale storage independently.  
   - Works well with Docker and containerized microservices.

   **Example schema in PostgreSQL:**
   ```sql
   CREATE TABLE images (
       id SERIAL PRIMARY KEY,
       user_id INT REFERENCES users(id),
       gesture_label VARCHAR(10),
       file_path TEXT NOT NULL,
       dataset_split VARCHAR(10), -- 'train', 'test', or 'validation'
       uploaded_at TIMESTAMP DEFAULT NOW()
   );



### 11.2 Storing Images Directly in the Database (BLOBs)

Images are stored as **binary large objects (BLOBs)** inside the database.

**Advantages:**

- Everything is in one system; backup includes images.

**Disadvantages:**

- Can significantly bloat the database.
- Slower read/write for large datasets.
- Harder to scale and serve images efficiently.

**Conclusion:** For our ASL Translator project, we recommend **option 1**: storing images externally (filesystem or cloud) and saving references in PostgreSQL.

---

### 11.3 Tracking Dataset Versions

Each training/testing dataset can be versioned in the database:

- Include a `dataset_version` column in the `images` table.
- Helps reproduce training runs and track which data was used for which model.

```sql
ALTER TABLE images ADD COLUMN dataset_version VARCHAR(20) DEFAULT 'v1';


Combine this with MLflow to log:

- Which dataset version was used for training each model.

- Which model version corresponds to which image data.

In [1]:
# mlflow.set_tag("dataset_version", "v1")
# mlflow.log_artifact("training_data_manifest.csv", artifact_path="datasets")

---

### 11.4 Metadata and Annotations

Besides the raw image path, we can store:

- Gesture label (for supervised training)

- Confidence scores (if predictions are logged)

- User ID (optional, for personalized datasets)

- Timestamps (for monitoring dataset growth)

- Dataset split (train, validation, test)

This ensures we can query images efficiently without touching the binary files themselves.

---
## 11.5 Integration with Microservices

Inference service: 
- Loads images via file paths for preprocessing and prediction.

Training service: 
- Reads image paths from the database for batch training.

Experiment tracking: 
- MLflow records dataset version and corresponding model version.

This setup keeps the database lean, works with Docker, and maintains reproducibility, which is essential for collaborative development.